In [169]:
import os, sys
sys.path.insert(1, os.getcwd()+'/..')
if (os.path.basename(os.getcwd()) == 'tutorials'):
    os.chdir('../..')

from src.msa import generate
from src.msa.visualization import plot
from src.msa.feature_extraction import features

from mspc_pca.pca import *
from mspc_pca.omeda import *
from mspc_pca.mspc import *
from mspc_pca.ckf import *
import mspc_pca.plot as pca_plot

import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec

In [170]:
%matplotlib qt

Let's generate a signal that features several different phenomena

In [171]:
# General parameters of the signal:
T = 60 #s. Duration in secods
sr = 100 # Sampling rate
n_samples = int(T*sr)

In [172]:
np.random.seed(123)
# Generate background noise as high frequency signals
F_noise = generate.frequencies(1000, 1e6, 100, sr)
A_noise = generate.amplitudes(1000, 0.0, sigma=0.001)
phi_noise = generate.phase(1000)
time, noise = generate.signal(F_noise, A_noise, sr,T, phi=phi_noise)
# Generate white noise
A = 2
white = np.random.rand(int(T*sr))*A*2 - A

# Generate pulses
pulse  = generate.pulse(1.2, .5, t0=7, t=time, f0=0.5, phi=0 )
pulse += generate.pulse(1.5, .4, t0=12,t=time, f0=1,phi=0 )
pulse += generate.pulse(1.4, .4, t0=42,t=time, f0=5,phi=0 )

# Generate stationary signal
F = generate.frequencies(10, 25, 0.1, sr)
A = generate.amplitudes(10, 0.003, sigma=0.001)
phi = generate.phase(10)
time, stationary = generate.signal(F, A, sr,T, phi=phi)

# Chirp sin pulse

chirp_sin  = generate.chirp_sin(time, 0.1,3)
chirp_sin += generate.chirp_sin(time, 0.01,7)
chirp_sin += generate.chirp_sin(time, 0.2,15)
chirp_sin += generate.chirp_sin(time, 0.003,33)
chirp_sin += generate.chirp_sin(time, 0.002,36)

# Chirp

chirp   = generate.chirp(time, A=1, f0=0, f1=50, t0=1,t1=60,method='linear',phi=0)*generate.decay(time, 0.0, 1)
chirp  += generate.chirp(time, A=1, f0=50, f1=0, t0=1,t1=60,method='linear',phi=0)*generate.decay(time, 0.0, 1)
chirp  += generate.chirp(time, A=1.5, f0=10, f1=15, t0=25,t1=30,method='linear',phi=0)*generate.decay(time, 0.1, 25)
chirp  += generate.chirp(time, A=2, f0=5, f1=5.5, t0=40,t1=41,method='linear',phi=0)*generate.decay(time, 0.2, 40)
chirp  += generate.chirp(time, A=3, f0=40, f1=10, t0=15,t1=30,method='linear',phi=0)*generate.decay(time, 0.5, 15)



# plot.signal(time, pulse)
signal = chirp + white #chirp_sin# + pulse + white + noise
# plot.signal(time, signal);

tr=None

# Use seismic data
import obspy
path = "/media/gsus/76C8A92EC8A8EE15/data/mseed/"#C7.PPMA..*.D.*.*"

files = ['*.PPMA..*.D.2021.213', '*.PPMA..*.D.2021.214', '*.PPMA..*.D.2021.215', '*.PPMA..*.D.2021.216', '*.PPMA..*.D.2021.217', '*.PPMA..*.D.2021.218', '*.PPMA..*.D.2021.219', '*.PPMA..*.D.2021.220', '*.PPMA..*.D.2021.221', '*.PPMA..*.D.2021.222', '*.PPMA..*.D.2021.223', '*.PPMA..*.D.2021.224', '*.PPMA..*.D.2021.225', '*.PPMA..*.D.2021.226', '*.PPMA..*.D.2021.227', '*.PPMA..*.D.2021.228', '*.PPMA..*.D.2021.229', '*.PPMA..*.D.2021.230', '*.PPMA..*.D.2021.231', '*.PPMA..*.D.2021.232', '*.PPMA..*.D.2021.233', '*.PPMA..*.D.2021.234', '*.PPMA..*.D.2021.235', '*.PPMA..*.D.2021.236', '*.PPMA..*.D.2021.237', '*.PPMA..*.D.2021.238', '*.PPMA..*.D.2021.239', '*.PPMA..*.D.2021.240', '*.PPMA..*.D.2021.241', '*.PPMA..*.D.2021.242', '*.PPMA..*.D.2021.243', '*.PPMA..*.D.2021.244', '*.PPMA..*.D.2021.245', '*.PPMA..*.D.2021.246', '*.PPMA..*.D.2021.247', '*.PPMA..*.D.2021.248', '*.PPMA..*.D.2021.249', '*.PPMA..*.D.2021.250', '*.PPMA..*.D.2021.251', '*.PPMA..*.D.2021.252', '*.PPMA..*.D.2021.253', '*.PPMA..*.D.2021.254', '*.PPMA..*.D.2021.255', '*.PPMA..*.D.2021.256', '*.PPMA..*.D.2021.257', '*.PPMA..*.D.2021.258', '*.PPMA..*.D.2021.259', '*.PPMA..*.D.2021.260', '*.PPMA..*.D.2021.261', '*.PPMA..*.D.2021.262', '*.PPMA..*.D.2021.263', '*.PPMA..*.D.2021.264', '*.PPMA..*.D.2021.265', '*.PPMA..*.D.2021.266', '*.PPMA..*.D.2021.267', '*.PPMA..*.D.2021.268', '*.PPMA..*.D.2021.269', '*.PPMA..*.D.2021.270', '*.PPMA..*.D.2021.271', '*.PPMA..*.D.2021.272', '*.PPMA..*.D.2021.273', '*.PPMA..*.D.2021.274', '*.PPMA..*.D.2021.275', '*.PPMA..*.D.2021.276', '*.PPMA..*.D.2021.277', '*.PPMA..*.D.2021.278', '*.PPMA..*.D.2021.279', '*.PPMA..*.D.2021.280', '*.PPMA..*.D.2021.281', '*.PPMA..*.D.2021.282', '*.PPMA..*.D.2021.283', '*.PPMA..*.D.2021.284', '*.PPMA..*.D.2021.285', '*.PPMA..*.D.2021.286', '*.PPMA..*.D.2021.287', '*.PPMA..*.D.2021.288', '*.PPMA..*.D.2021.289', '*.PPMA..*.D.2021.290', '*.PPMA..*.D.2021.291', '*.PPMA..*.D.2021.292', '*.PPMA..*.D.2021.293', '*.PPMA..*.D.2021.294', '*.PPMA..*.D.2021.295', '*.PPMA..*.D.2021.296', '*.PPMA..*.D.2021.297', '*.PPMA..*.D.2021.298', '*.PPMA..*.D.2021.299', '*.PPMA..*.D.2021.300', '*.PPMA..*.D.2021.301', '*.PPMA..*.D.2021.302', '*.PPMA..*.D.2021.303', '*.PPMA..*.D.2021.304', '*.PPMA..*.D.2021.305', '*.PPMA..*.D.2021.306', '*.PPMA..*.D.2021.307', '*.PPMA..*.D.2021.308', '*.PPMA..*.D.2021.309', '*.PPMA..*.D.2021.310', '*.PPMA..*.D.2021.311', '*.PPMA..*.D.2021.312', '*.PPMA..*.D.2021.313', '*.PPMA..*.D.2021.314', '*.PPMA..*.D.2021.315', '*.PPMA..*.D.2021.316', '*.PPMA..*.D.2021.317', '*.PPMA..*.D.2021.318', '*.PPMA..*.D.2021.319', '*.PPMA..*.D.2021.320', '*.PPMA..*.D.2021.321', '*.PPMA..*.D.2021.322', '*.PPMA..*.D.2021.323', '*.PPMA..*.D.2021.324', '*.PPMA..*.D.2021.325', '*.PPMA..*.D.2021.326', '*.PPMA..*.D.2021.327', '*.PPMA..*.D.2021.328', '*.PPMA..*.D.2021.329', '*.PPMA..*.D.2021.330', '*.PPMA..*.D.2021.331', '*.PPMA..*.D.2021.332', '*.PPMA..*.D.2021.333', '*.PPMA..*.D.2021.334', '*.PPMA..*.D.2021.335', '*.PPMA..*.D.2021.336', '*.PPMA..*.D.2021.337', '*.PPMA..*.D.2021.338', '*.PPMA..*.D.2021.339', '*.PPMA..*.D.2021.340', '*.PPMA..*.D.2021.341', '*.PPMA..*.D.2021.342', '*.PPMA..*.D.2021.343', '*.PPMA..*.D.2021.344', '*.PPMA..*.D.2021.345', '*.PPMA..*.D.2021.346', '*.PPMA..*.D.2021.347', '*.PPMA..*.D.2021.348', '*.PPMA..*.D.2021.349', '*.PPMA..*.D.2021.350', '*.PPMA..*.D.2021.351', '*.PPMA..*.D.2021.352', '*.PPMA..*.D.2021.353', '*.PPMA..*.D.2021.354', '*.PPMA..*.D.2021.355', '*.PPMA..*.D.2021.356', '*.PPMA..*.D.2021.357', '*.PPMA..*.D.2021.358', '*.PPMA..*.D.2021.359', '*.PPMA..*.D.2021.360', '*.PPMA..*.D.2021.361', '*.PPMA..*.D.2021.362', '*.PPMA..*.D.2021.363', '*.PPMA..*.D.2021.364', '*.PPMA..*.D.2021.365', '*.PPMA..*.D.2022.001', '*.PPMA..*.D.2022.002', '*.PPMA..*.D.2022.003', '*.PPMA..*.D.2022.004', '*.PPMA..*.D.2022.005', '*.PPMA..*.D.2022.006', '*.PPMA..*.D.2022.007', '*.PPMA..*.D.2022.008', '*.PPMA..*.D.2022.009', '*.PPMA..*.D.2022.010', '*.PPMA..*.D.2022.011', '*.PPMA..*.D.2022.012', '*.PPMA..*.D.2022.013', '*.PPMA..*.D.2022.014', '*.PPMA..*.D.2022.015', '*.PPMA..*.D.2022.016', '*.PPMA..*.D.2022.017', '*.PPMA..*.D.2022.018', '*.PPMA..*.D.2022.019', '*.PPMA..*.D.2022.020', '*.PPMA..*.D.2022.021', '*.PPMA..*.D.2022.022', '*.PPMA..*.D.2022.023', '*.PPMA..*.D.2022.024', '*.PPMA..*.D.2022.025', '*.PPMA..*.D.2022.026', '*.PPMA..*.D.2022.027', '*.PPMA..*.D.2022.028', '*.PPMA..*.D.2022.029', '*.PPMA..*.D.2022.030', '*.PPMA..*.D.2022.031', '*.PPMA..*.D.2022.032']
files = files[49:50]
title = ""
filepath = []

for file in files:
    filepath.append(path+file)
    
print(f"Reading files ...")

if type(filepath)==str:
    st = obspy.read(filepath)
else: 
    st = obspy.read(filepath[0])
    for i in range(1,len(filepath)):
        try:
            st+= obspy.read(filepath[i])
            print(f"Reading {filepath[i]} data...")
        except:print(f"{filepath[i]} not read.")

print(st.__str__(extended=True))

print(f"Merging {len(st)} traces ...")
st.merge()


tr = st[1]
signal = tr.data
sr = tr.stats.sampling_rate

In [173]:
# Parameters
sr = sr # fixed by out sensors
window = 1#s. Higher window will lead to better freq. resolution
shift  = .1 #s. Lower shift leads to better time resolution 
win_samples = int(sr*window)
hop = int(sr*shift)

print(f"Time resolution: {shift}s")
print(f"Frequency resolution: {1/window}Hz")

times, freqs, Sxx = features.spectrogram(signal, sr, win_samples, hop,"boxcar",detrend='linear')

freqs_label = [f"{round(x, 2)}Hz" for x in freqs]
times_label = [f"{round(x, 2)}s" for x in times]

if tr is not None:
    starttime = tr.stats.starttime.datetime
    endtime = tr.stats.endtime.datetime

time = np.linspace(times[0], times[-1],len(signal))

Sxx.shape


Time resolution: 0.1s
Frequency resolution: 1.0Hz


(51, 609)

In [174]:
import scipy.io
data={
    'spectrogram': Sxx,
    'times': times,
    'freqs': freqs,
}
scipy.io.savemat('./external/tutorials/data.mat', data)

In [175]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

autoscale = False

data = Sxx.T
scaler = StandardScaler(with_std=autoscale)
data_norm = scaler.fit_transform(data)
pca_plot.var_pca(data, 5, autoscale);


In [176]:

pca = PCA(n_components=4)

fig = plt.figure(figsize=(15, 6)) 
gs = gridspec.GridSpec(2, 2, width_ratios=[2, 3]) 

ax1 = fig.add_subplot(gs[0, 0])  # Fila 0, Columna 0
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)  # Fila 1, Columna 0
ax3 = fig.add_subplot(gs[:, 1])  # Todas las filas de columna 1

if tr is not None:
    timeUTC = np.linspace(starttime, endtime, num= len(times), dtype='datetime64[m]').astype("datetime64[m]")
    times_label = timeUTC
plot.signal(time, signal, ax=ax1)
plot.spectrogram(times, freqs, Sxx, logscale=False, ylim=(0,50), ax=ax2)
pca_plot.biplot(data_norm,pca,1,2,loading_labels=freqs_label, score_labels=times_label, label_dist=0.1, ax=ax3);

ax1.set_ylabel("Amplitude")
ax2.set_ylabel("Frequency (Hz)")
ax2.set_xlabel("Time")
f_res = freqs[1]-freqs[0]
ax3.set_title(f"Time resolution: {shift}s | Frequency resolution: {round(f_res, 2)}Hz")

if tr is not None:
    n_windows = len(times)
    n_ticks:int = 3
    skip = round(n_windows/(n_ticks)) 
    locs = times[0::skip]
    timeUTC = timeUTC[0::skip]
    date_formatter = mdates.DateFormatter('%Y/%m/%d %H:%M')
    formatted_labels = [date_formatter(mdates.date2num(t)) for t in timeUTC]
    ax2.set_xticks(locs[:], labels=formatted_labels[:],fontsize=12)

plt.setp(ax1.get_xticklabels(), visible=False)
plt.tight_layout()
plt.show() 